In [21]:
using BitemporalPostgres
using LifeInsuranceDataModel, LifeInsuranceProduct
using SearchLight
using SearchLightPostgreSQL
using TimeZones
ENV["SEARCHLIGHT_USERNAME"] = "mf"
ENV["SEARCHLIGHT_PASSWORD"] = "mf"
SearchLight.Configuration.load() |> SearchLight.connect

SearchLight.query("DROP SCHEMA public CASCADE")
SearchLight.query("CREATE SCHEMA public")
LifeInsuranceDataModel.load_model()

┌ Info: DROP SCHEMA public CASCADE
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  drop cascades to 37 other objects
DETAIL:  drop cascades to table schema_migrations
drop cascades to table histories
drop cascades to table versions
drop cascades to table workflows
drop cascades to table testtstzranges
drop cascades to table validityintervals
drop cascades to table testdummycomponents
drop cascades to table testdummycomponentrevisions
drop cascades to table testdummysubcomponents
drop cascades to table testdummysubcomponentrevisions
drop cascades to extension btree_gist
drop cascades to function f_bitempranges()
drop cascades to function f_versionrange()
drop cascades to table contractpartnerroles
drop cascades to table tariffitempartnerroles
drop cascades to table tariffitemroles
drop cascades to table productpartroles
drop cascades to table contracts
drop cascades to table contractrevisions
drop cascades

┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/vpxjS/src/LifeInsuranceDataModel.jl:573
┌ Info: CREATE TABLE schema_migrations (version varchar(30))
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: Created table schema_migrations
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:306
┌ Info: CREATE TABLE histories (id BIGSERIAL PRIMARY KEY, dummy INTEGER )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE versions (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE workflows (id BIGSERIAL PRIMARY KEY, ref_history B

┌ Info: CREATE TABLE testdummyComponents (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE testdummyComponentRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES testdummyComponents(id), ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE testdummySubComponents (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES testdummyComponents(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_super BIGINT REFERENCES his

┌ Info: CREATE OR REPLACE FUNCTION f_bitempranges ()
│ RETURNS trigger AS
│ $$
│      DECLARE
│ 
│      BEGIN
│           RAISE NOTICE 'NEW: %', NEW;
│           NEW.tsrworld := tstzrange(NEW.tsworld_validfrom,NEW.tsworld_invalidfrom,'[)');
│           NEW.tsrdb := tstzrange(NEW.tsdb_validfrom,NEW.tsdb_invalidfrom,'[)');
│           RETURN NEW;
│      END;
│ $$ LANGUAGE 'plpgsql';
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TRIGGER versions_trig
│ BEFORE INSERT OR UPDATE ON validityIntervals
│ FOR EACH ROW EXECUTE PROCEDURE f_bitempranges();
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE OR REPLACE FUNCTION f_versionrange ()
│ RETURNS trigger AS
│ $$
│      DECLARE
│ 
│      BEGIN
│           RAISE NOTICE 'NEW: %', NEW;
│           NEW.ref_valid := int8range(NEW.ref_validfrom,NEW.ref_invalidfrom,'[)');
│    

┌ Info: CREATE TABLE tariffItemPartnerRoles (id BIGSERIAL PRIMARY KEY, domain VARCHAR , value VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE TariffItemRoles (id BIGSERIAL PRIMARY KEY, domain VARCHAR , value VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE ProductPartRoles (id BIGSERIAL PRIMARY KEY, domain VARCHAR , value VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE contracts (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


...

┌ Info: ALTER TABLE productitemRevisions
│ ADD CONSTRAINT pi_versionrange EXCLUDE USING GIST (ref_component WITH =, ref_valid WITH &&)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE partners (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE partnerRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES partners(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR , date_of_birth DATE , sex VARCHAR , smoker BOOLEAN )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/Searc

┌ Info: CREATE TABLE contractPartnerRefRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES contractPartnerRefs(id) ON DELETE CASCADE, ref_role BIGINT REFERENCES contractpartnerroles(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR , ref_partner BIGINT REFERENCES partners(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TRIGGER cprr_versions_trig
│ BEFORE INSERT OR UPDATE ON contractPartnerRefRevisions
│ FOR EACH ROW EXECUTE PROCEDURE f_versionrange();
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: ALTER TABLE contractPartnerRefRevisions
│ ADD CONSTRAINT cprr_versionrange EXCLUDE USING GIST (ref_component WITH =, ref_

┌ Info: ALTER TABLE TariffItemRevisions
│ ADD CONSTRAINT pitrr_versionrange EXCLUDE USING GIST (ref_component WITH =, ref_valid WITH &&)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE TariffItemPartnerRefs (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_super BIGINT REFERENCES tariffitems(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE TariffItemPartnerRefRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES TariffItemPartnerRefs(id) ON DELETE CASCADE, ref_role BIGINT REFERENCES tariffitempartnerroles(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELE

┌ Info: INSERT INTO contractpartnerroles ( "domain", "value" ) VALUES ( E'ContractPartner', E'Premium Payer' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "contractpartnerroles"."id" AS "contractpartnerroles_id", "contractpartnerroles"."domain" AS "contractpartnerroles_domain", "contractpartnerroles"."value" AS "contractpartnerroles_value" FROM "contractpartnerroles" WHERE "id" = 2 ORDER BY contractpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO tariffitempartnerroles ( "domain", "value" ) VALUES ( E'TariffItemPartner', E'Insured Person' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitempartnerroles"."id" AS "tariffitempartnerroles_id", "tariffitempartnerroles".

1×4 Matrix{ProductPartRole}:
 ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 1                    |
| value::String  | Main Coverage - Life |
  …  ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 4                    |
| value::String  | Profit participation |


In [22]:
using LifeInsuranceDataModel, LifeInsuranceProduct, SearchLight
import SearchLight: Serializer.serialize

tif = get_tariff_interface(Val(1))
tif.parameters


LifeRiskTariff = create_tariff("Life Risk Insurance", 1, "", serialize(tif.parameters))
ProfitParticipationTariff = create_tariff(
    "Profit participation", 2,
    "", "{}"
)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

NOTICE:  NEW: (1,1,1,"2014-05-30 21:00:01.001+00","2038-01-19 03:14:06.999+00","2023-03-27 07:15:25.45+00","2038-01-19 03:14:06.999+00","[""2014-05-30 21:00:01.001+00"",""2038-01-19 03:14:06.999+00"")","[""2023-03-27 07:15:25.428+00"",""2038-01-19 03:14:06.999+00"")",1)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE "id" = 1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/pa

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'2' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T21:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE validity

2

In [23]:
cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end

p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=LifeRiskTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

┌ Info: SELECT "contractpartnerroles"."id" AS "contractpartnerroles_id", "contractpartnerroles"."domain" AS "contractpartnerroles_domain", "contractpartnerroles"."value" AS "contractpartnerroles_value" FROM "contractpartnerroles" ORDER BY contractpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitempartnerroles"."id" AS "tariffitempartnerroles_id", "tariffitempartnerroles"."domain" AS "tariffitempartnerroles_domain", "tariffitempartnerroles"."value" AS "tariffitempartnerroles_value" FROM "tariffitempartnerroles" ORDER BY tariffitempartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 3 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'3' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 3 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'3' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2023-04-02T21:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE validity

In [24]:
prs = prsection(1, now(tz"UTC"), ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"), 0)

┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/vpxjS/src/LifeInsuranceDataModel.jl:573
┌ Info: SELECT "products"."id" AS "products_id", "products"."ref_history" AS "products_ref_history", "products"."ref_version" AS "products_ref_version" FROM "products" WHERE id=E'1' ORDER BY products.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_inval

┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" WHERE ref_history=E'2' ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffrevisions"."id" AS "tariffrevisions_id", "tariffrevisions"."ref_component" AS "tariffrevisions_ref_component", "tariffrevisions"."ref_validfrom" AS "tariffrevisions_ref_validfrom", "tariffrevisions"."ref_invalidfrom" AS "tariffrevisions_ref_invalidfrom", "tariffrevisions"."description" AS "tariffrevisions_description", "tariffrevisions"."interface_id" AS "tariffrevisions_interface_id", "tariffrevisions"."parameters" AS "tariffrevisions_parameters", "tariffrevisions"."mortality_table" AS "tariffrevisions_mortality_table" FROM "tariffrevisions" WHERE ref_component=E'2' and ref_valid @> BIGINT E'2' ORDER BY tariffrevisions.id ASC
└ @ SearchLigh

ProductSection(ZonedDateTime(2023, 3, 27, 7, 15, 25, 988, tz"UTC"), ZonedDateTime(2023, 3, 27, 7, 15, 25, 988, tz"UTC"), 9007199254740991, 9007199254740991, ProductRevision
| KEY                   | VALUE            |
|-----------------------|------------------|
| description::String   | Life Risk        |
| id::DbId              | 1                |
| ref_component::DbId   | 1                |
| ref_invalidfrom::DbId | 9007199254740991 |
| ref_validfrom::DbId   | 3                |
, ProductPartSection[ProductPartSection(ProductPartRevision
| KEY                   | VALUE                |
|-----------------------|----------------------|
| description::String   | Main Coverage - Life |
| id::DbId              | 1                    |
| ref_component::DbId   | 1                    |
| ref_invalidfrom::DbId | 9007199254740991     |
| ref_role::DbId        | 1                    |
| ref_tariff::DbId      | 1                    |
| ref_validfrom::DbId   | 3                    |
, TariffSec

In [25]:

# create partner 1
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1",sex="f",smoker=false)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)
Partner1 = p.id

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 4 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 4 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


1

In [26]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=Partner1,
    ref_role=cpRole["Policy Holder"],
    description="policiyholder ref properties",
)
create_subcomponent!(c, cpr, cprr, w1)
# pi 1
LifeRiskProduct = find(Product, SQLWhereExpression("id=?", 1))[1].id.value
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=1,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    LifeRiskProduct,
    Partner1.value,
    PartnerRole,
)
commit_workflow!(w1)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 5 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'5' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 5 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: INSERT INTO contractpartnerrefs ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'5', E'9007199254740991', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "contractpartnerrefs"."id" AS "contractpartnerrefs_id", "contractpartnerrefs"."ref_history" AS "contractpartnerrefs_ref_history", "contractpartnerrefs"."ref_version" AS "contractpartnerrefs_ref_version", "contractpartnerrefs"."ref_super" AS "contractpartnerrefs_ref_super" FROM "contractpartnerrefs" WHERE "id" = 1 ORDER BY contractpartnerrefs.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO contractpartnerrefrevisions ( "ref_component", "ref_role", "ref_validfrom", "ref_invalidfrom", "description", "ref_partner" ) VALUES ( E'1', E'1', E'5', E'9007199254740991', E'policiyholder ref properties', E'1' ) RETURNING id
└ @

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO productitems ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'5', E'9007199254740991', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "productitems"."id" AS "productitems_id", "productitems"."ref_history" AS "productitems_ref_history", "productitems"."ref_version" AS "productitems_ref_version", "productitems"."ref_super" AS "productitems_ref_super" FROM "productitems" WHERE "id" = 1 ORDER BY productitems.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: INSERT INTO productitemrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "ref_product", "description" ) VALUES ( E'1', E'5', E'9007199254740991', E'1', E'from contract creation' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (1,1,1,5,9007199254740991,,"from contract creation")
┌ Info: SELECT "productitemrevisions"."id" AS "productitemrevisions_id", "productitemrevisions"."ref_component" AS "productitemrevisions_ref_component", "productitemrevisions"."ref_validfrom" AS "productitemrevisions_ref_validfrom", "productitemrevisions"."ref_invalidfrom" AS "productitemrevisions_ref_invalidfrom", "productitemrevisions"."ref_product" AS "productitemrevisions_ref_product", "productitemrevisions"."description" AS "productitemrevisions_description" FROM "productitemrevisions" WHERE "id" = 1 ORDER BY productitemrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_dep

...

TariffItemRevision(ref_validfrom::DbId=5,ref_invalidfrom::DbId=9007199254740991,parameters::String={"mortality_tables":{"f":{"nonsmoker":"1986-92 CIA – Female Nonsmoker, ANB","smoker":"1986-92 CIA – Female Smoker, ANB"},"m":{"nonsmoker":"1986-92 CIA – Male Nonsmoker, ANB","smoker":"1986-92 CIA – Male Smoker, ANB"}}},id::DbId=1,ref_component::DbId=1,ref_role::DbId=1,ref_tariff::DbId=1,description::String=Main Coverage - Life)


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=2 ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffrevisions"."id" AS "tariffrevisions_id", "tariffrevisions"."ref_component" AS "tariffrevisions_ref_component", "tariffrevisions"."ref_validfrom" AS "tariffrevisions_ref_validfrom", "tariffrevisions"."ref_invalidfrom" AS "tariffrevisio

TariffItemPartnerRefRevision(ref_validfrom::DbId=5,ref_invalidfrom::DbId=9007199254740991,id::DbId=1,ref_partner::DbId=1,ref_component::DbId=1,ref_role::DbId=1,description::String=)
2
Profit participation Life Risk
TariffItemRevision(ref_validfrom::DbId=5,ref_invalidfrom::DbId=9007199254740991,parameters::String={},id::DbId=2,ref_component::DbId=2,ref_role::DbId=4,ref_tariff::DbId=2,description::String=Profit participation Life Risk)
TariffItemPartnerRefRevision(ref_validfrom::DbId=5,ref_invalidfrom::DbId=9007199254740991,id::DbId=2,ref_partner::DbId=1,ref_component::DbId=2,ref_role::DbId=1,description::String=)


┌ Info: UPDATE workflows SET  "id" = E'5', "ref_history" = E'5', "ref_version" = E'5', "tsdb_validfrom" = E'2023-03-27T07:15:33.826+00:00', "tsw_validfrom" = E'2014-05-30T21:00:01.001+00:00', "is_committed" = 1, "type_of_entity" = E'Contract' WHERE workflows.id = '5' RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed", "workflows"."type_of_entity" AS "workflows_type_of_entity" FROM "workflows" WHERE "id" = 5 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: COMMIT
└ @ SearchLi

In [27]:
cid = 1
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]


┌ Info: SELECT "contracts"."id" AS "contracts_id", "contracts"."ref_history" AS "contracts_ref_history", "contracts"."ref_version" AS "contracts_ref_version" FROM "contracts" WHERE id =1 ORDER BY contracts.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history=E'5' ORDER BY validit

┌ Info: SELECT "contractpartnerrefrevisions"."id" AS "contractpartnerrefrevisions_id", "contractpartnerrefrevisions"."ref_component" AS "contractpartnerrefrevisions_ref_component", "contractpartnerrefrevisions"."ref_role" AS "contractpartnerrefrevisions_ref_role", "contractpartnerrefrevisions"."ref_validfrom" AS "contractpartnerrefrevisions_ref_validfrom", "contractpartnerrefrevisions"."ref_invalidfrom" AS "contractpartnerrefrevisions_ref_invalidfrom", "contractpartnerrefrevisions"."description" AS "contractpartnerrefrevisions_description", "contractpartnerrefrevisions"."ref_partner" AS "contractpartnerrefrevisions_ref_partner" FROM "contractpartnerrefrevisions" WHERE ref_component=E'1' and ref_valid @> BIGINT E'5' ORDER BY contractpartnerrefrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/

┌ Info: SELECT "productitems"."id" AS "productitems_id", "productitems"."ref_history" AS "productitems_ref_history", "productitems"."ref_version" AS "productitems_ref_version", "productitems"."ref_super" AS "productitems_ref_super" FROM "productitems" WHERE ref_history = BIGINT E'5' ORDER BY productitems.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productitemrevisions"."id" AS "productitemrevisions_id", "productitemrevisions"."ref_component" AS "productitemrevisions_ref_component", "productitemrevisions"."ref_validfrom" AS "productitemrevisions_ref_validfrom", "productitemrevisions"."ref_invalidfrom" AS "productitemrevisions_ref_invalidfrom", "productitemrevisions"."ref_product" AS "productitemrevisions_ref_product", "productitemrevisions"."description" AS "productitemrevisions_description" FROM "productitemrevisions" WHERE ref_component=E'1' and ref_valid @> BIGINT E'5' ORDER BY product

┌ Info: SELECT "tariffitems"."id" AS "tariffitems_id", "tariffitems"."ref_history" AS "tariffitems_ref_history", "tariffitems"."ref_version" AS "tariffitems_ref_version", "tariffitems"."ref_super" AS "tariffitems_ref_super" FROM "tariffitems" WHERE ref_history = BIGINT E'5' and ref_super = BIGINT E'1' ORDER BY tariffitems.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitemrevisions"."id" AS "tariffitemrevisions_id", "tariffitemrevisions"."ref_component" AS "tariffitemrevisions_ref_component", "tariffitemrevisions"."ref_role" AS "tariffitemrevisions_ref_role", "tariffitemrevisions"."ref_validfrom" AS "tariffitemrevisions_ref_validfrom", "tariffitemrevisions"."ref_invalidfrom" AS "tariffitemrevisions_ref_invalidfrom", "tariffitemrevisions"."description" AS "tariffitemrevisions_description", "tariffitemrevisions"."ref_tariff" AS "tariffitemrevisions_ref_tariff", "tariffitemrevisions"."pa

┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/vpxjS/src/LifeInsuranceDataModel.jl:573
┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" WHERE id=E'1' ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom",

┌ Info: SELECT "tariffitempartnerrefrevisions"."id" AS "tariffitempartnerrefrevisions_id", "tariffitempartnerrefrevisions"."ref_component" AS "tariffitempartnerrefrevisions_ref_component", "tariffitempartnerrefrevisions"."ref_role" AS "tariffitempartnerrefrevisions_ref_role", "tariffitempartnerrefrevisions"."ref_validfrom" AS "tariffitempartnerrefrevisions_ref_validfrom", "tariffitempartnerrefrevisions"."ref_invalidfrom" AS "tariffitempartnerrefrevisions_ref_invalidfrom", "tariffitempartnerrefrevisions"."description" AS "tariffitempartnerrefrevisions_description", "tariffitempartnerrefrevisions"."ref_partner" AS "tariffitempartnerrefrevisions_ref_partner" FROM "tariffitempartnerrefrevisions" WHERE ref_component=E'1' and ref_valid @> BIGINT E'5' ORDER BY tariffitempartnerrefrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/

┌ Info: SELECT "tariffpartnerroles"."id" AS "tariffpartnerroles_id", "tariffpartnerroles"."ref_history" AS "tariffpartnerroles_ref_history", "tariffpartnerroles"."ref_version" AS "tariffpartnerroles_ref_version", "tariffpartnerroles"."ref_super" AS "tariffpartnerroles_ref_super" FROM "tariffpartnerroles" WHERE ref_super=E'2' ORDER BY tariffpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffpartnerrolerevisions"."id" AS "tariffpartnerrolerevisions_id", "tariffpartnerrolerevisions"."ref_component" AS "tariffpartnerrolerevisions_ref_component", "tariffpartnerrolerevisions"."ref_validfrom" AS "tariffpartnerrolerevisions_ref_validfrom", "tariffpartnerrolerevisions"."ref_invalidfrom" AS "tariffpartnerrolerevisions_ref_invalidfrom", "tariffpartnerrolerevisions"."ref_role" AS "tariffpartnerrolerevisions_ref_role" FROM "tariffpartnerrolerevisions" WHERE ref_component=E'2' and ref_vali

TariffItemSection(TariffItemTariffReference(TariffItemRevision
| KEY                   | VALUE                                                                                                                                                                                                                      |
|-----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| description::String   | Main Coverage - Life                                                                                                                                                                                                       |
| id::DbId              | 1                                                                                                                                                                         

In [28]:
using JSON, LifeInsuranceProduct
tariffparms = JSON.parse(ti.tariff_ref.rev.parameters)
mts = tariffparms["mortality_tables"]
interface_id = ti.tariff_ref.ref.revision.interface_id
tgt = get_tariff_interface(Val(interface_id)).calls["calculation_target"]
tgt["selected"] = "ä"
parms = tgt[tgt["selected"]]
parms["m"] = 10
parms["n"] = 20
parms["frequency"] = 2
parms["begin"] = string(Date("2023-04-01"))

parms
dob = ti.partner_refs[1].ref.revision.date_of_birth
smoker = ti.partner_refs[1].ref.revision.smoker ? "smoker" : "nonsmoker"
sex = ti.partner_refs[1].ref.revision.sex

issue_age = LifeInsuranceProduct.insurance_age(dob, Date(parms["begin"]))


life = SingleLife(
    mortality=MortalityTables.table(mts[sex][smoker]).select[issue_age])

yield = Yields.Constant(0.0125)      # Using a flat

lc = LifeContingency(life,yield)  # LifeContingenc


r1 = ä(lc)
r2 = ä(lc,1)
r3 = ä(lc, 2)
r4 = ä(lc, 30)

println("r1")
println(r1)
println("r2")
println(r2)
println("r3")
println(r3)
println("r4")
println(r4)


r1
42.531166976465144
r2


UndefVarError: UndefVarError: r2 not defined